## Features:

### Client:

- Client_id: Unique id for client
- District: District where the client is
- Client_catg: Category client belongs to
- Region: Area where the client is
- Creation_date: Date client joined
- Target: fraud:1 , not fraud: 0


### Invoice data

- Client_id: Unique id for the client
- Invoice_date: Date of the invoice
- Tarif_type: Type of tax
- Counter_number:
- Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
- Counter_code:
- Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
- Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
- Consommation_level_1: Consumption_level_1
- Consommation_level_2: Consumption_level_2
- Consommation_level_3: Consumption_level_3
- Consommation_level_4: Consumption_level_4
- Old_index: Old index
- New_index: New index
- Months_number: Month number
- Counter_type: Type of counter


## Some findings

- the "test" = "competition" set (no targets there). Therefore must split the "train" set into train/test sets

- must not aggregate to make a shorter table with customers. Instead predict on TRANSACTIONS. (df.groupby('client_id').nunique())

- the proportion of positives is higher in the merged "transactions" table than the proportion of positives in the "clients" table (0.06 / 0.08).  So you can treat that as "perturbation" of positives in order to increase the number of positives (where they are scarse). I.e. this is one more argument in the favour of predicting on transactions  and then aggregating them to get a prediction for a particular customer.

- the 'months_number' column does not contain actual months. These values do not correspond to the 'creation_date' or 'invoice_date' columns.  Either keep this columns without any transformation or scaling or delete it completeley. Because the test set contains this kidn of wierd values too.

- the features  ['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4']   are not very promising (in tearms of building univariate logistic regression on them)

- columns 'counter_statue'  is supposed to be integers [0-5] but is of mixed type (object) with some bogus values.  Convert to int, drop the rows with values > 5, because the test set doesnt have any bad values in this column - only the valid integers from 0 to 5

- search for a decent baseline model didn't give decent results. Try non-deterministic baseline model based on the prior (i.e. the proportion of positives in the population)

- eventually agreed to predict transactions (and not fraudulent clients)

- rule-based baseline model on two rules (2005, higher consumption)


In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.metrics import fbeta_score, confusion_matrix, recall_score, precision_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score

from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import SMOTENC

In [2]:
# TODO: random seed
...

In [3]:
# feature preprocessing functions

def preprocess(feature, data):
    functions = {'counter_statue': preprocess_counter_statue}
    feature = feature if type(feature) is str else data.name if type(data) is pd.Series else data.columns[feature]
    function = functions[feature]
    return function(data)

    
# preprocess 'counter_statue'
def preprocess_counter_statue(data):
    col = 'counter_statue'
    sr = data[col].astype(str)
    mask = sr.isin(list("012345"))
    sr[~mask] = sr[mask].mode().values[0]
    data[col] = sr.astype(int)


In [4]:
path1 = "data/train/client_train.csv"
path2 = "data/train/invoice_train.csv"

path3 = "data/test/client_test.csv"
path4 = "data/test/invoice_test.csv"

In [5]:
# load the data

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2, low_memory=False)     # low_memory=False

df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)

In [6]:
# join tables

# data from the "train" folder (will have to be split into train/test)
df_entire = df1.merge(df2, left_on='client_id', right_on='client_id', how='outer')

# data from the "test" folder (doesn't contain targets)
df_test_zindi = df3.merge(df4, left_on='client_id', right_on='client_id', how='outer')


In [7]:
# converts all values to int, fills bad values with the mode
preprocess('counter_statue', df_entire)

In [8]:
# quick feature engineering 

df_entire['year_created'] = pd.to_datetime(df_entire['creation_date'],
                                           format="%d/%m/%Y").dt.year
dates = pd.to_datetime(df_entire['invoice_date'])
df_entire['invoice_year'] = dates.dt.year
df_entire['invoice_month'] = dates.dt.month
df_entire['invoice_weekday'] = dates.dt.weekday

In [9]:
# drop the observations before 2005

YEAR = 2005
df_entire.drop(df_entire.index[df_entire['invoice_year'] < YEAR], axis=0, inplace=True)

# reset index
df_entire.reset_index(drop=True, inplace=True)

In [10]:
# enlabel 'counter_type'

df_entire['counter_type'], _ = pd.factorize(df_entire['counter_type'])

In [11]:
# drop these columns

cols = ['disrict', 'client_id', 'creation_date', 'invoice_date', 'old_index',
       'months_number', ]
df_entire.drop(cols, axis=1, inplace=True)

In [12]:
# feature engineering: interaction between the 4 features
# takes a long time     # if product=0 then target=0 ?  experiment with this
"""
df_entire['mult'] = df_entire[['consommation_level_1', 'consommation_level_2',
                 'consommation_level_3', 'consommation_level_4']].apply(np.multiply.reduce, axis=1)
"""

"\ndf_entire['mult'] = df_entire[['consommation_level_1', 'consommation_level_2',\n                 'consommation_level_3', 'consommation_level_4']].apply(np.multiply.reduce, axis=1)\n"

## Split the data

In [13]:
df_X = df_entire.drop('target', axis=1)
sr_y = df_entire['target']

df_train, df_test, y_train, y_test = train_test_split(df_X, sr_y, test_size=0.2, stratify=sr_y)

Xtrain = df_train.values
Xtest = df_test.values
ytrain = y_train.values
ytest = y_test.values


## Upsample data with SMOTE

In [14]:
categoricals = [col for col in df_train.columns if df_train[col].nunique() < 20]
non_cats = [col for col in df_train.columns if df_train[col].nunique() >= 20]

nd_cats_encoded = OrdinalEncoder().fit_transform(df_train[categoricals]).astype(int)

df_train_encoded = pd.concat([pd.DataFrame(nd_cats_encoded, 
                                           columns=categoricals,
                                          index=df_train.index), 
                              df_train[non_cats]], axis=1)

df_train_encoded.head()

,client_catg,tarif_type,counter_statue,reading_remarque,counter_coefficient,counter_type,invoice_year,invoice_month,invoice_weekday,region,counter_number,counter_code,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,new_index,year_created
8948,0,3,0,4,1,0,1,3,4,306,676817,203,499,0,0,0,3034,2002
3288655,0,2,0,1,1,0,1,7,1,101,8517618,202,200,418,0,0,26902,1977
2820567,0,3,0,3,1,0,12,6,1,104,262968,207,0,0,0,0,24941,2016
4069141,0,13,0,1,1,1,11,2,1,101,308513,5,225,0,0,0,2171,2004
4142867,0,3,0,1,1,0,5,10,1,303,1667,207,352,0,0,0,12513,2002


In [15]:
sm = SMOTENC(categorical_features=list(range(len(categoricals))),
             k_neighbors=5,
            sampling_strategy=0.5)
df_train_smote, y_train_smote = sm.fit_resample(df_train_encoded, y_train)

df_train_smote.tail(15)

,client_catg,tarif_type,counter_statue,reading_remarque,counter_coefficient,counter_type,invoice_year,invoice_month,invoice_weekday,region,counter_number,counter_code,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,new_index,year_created
6562423,0,3,0,1,1,0,9,2,1,106,98267,413,375,32,42,0,27719,2003
6562424,2,15,0,4,1,1,12,1,3,312,5234894,25,1382,0,0,0,125279,1996
6562425,0,2,0,1,1,0,5,11,5,101,8566480,202,175,21,0,0,34117,1977
6562426,0,3,0,1,1,0,1,9,5,219,62681,413,402,0,0,0,12387,1988
6562427,0,13,0,1,1,1,7,4,1,311,73263,5,277,0,0,0,3718,1992
6562428,0,13,0,1,1,1,2,3,0,101,47971,5,0,0,0,0,0,1992
6562429,2,15,0,4,1,1,10,1,4,101,3677810,25,168,0,0,0,415125,2007
6562430,0,3,0,1,1,0,3,0,1,311,239664,203,505,0,0,0,33757,1990
6562431,0,7,0,1,1,0,6,4,4,303,193124,202,274,0,0,0,19056,1981
6562432,0,3,0,3,1,0,11,3,0,101,482672,203,40,0,0,0,35129,1991


In [17]:
y_train_smote.sum() / len(y_train_smote)

0.5

In [19]:
df_train_smote_y = pd.concat([df_train_smote, y_train_smote.astype(int)], axis=1)
df_train_smote_y.head()

,client_catg,tarif_type,counter_statue,reading_remarque,counter_coefficient,counter_type,invoice_year,invoice_month,invoice_weekday,region,counter_number,counter_code,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,new_index,year_created,target
0,0,3,0,4,1,0,1,3,4,306,676817,203,499,0,0,0,3034,2002,0
1,0,2,0,1,1,0,1,7,1,101,8517618,202,200,418,0,0,26902,1977,0
2,0,3,0,3,1,0,12,6,1,104,262968,207,0,0,0,0,24941,2016,0
3,0,13,0,1,1,1,11,2,1,101,308513,5,225,0,0,0,2171,2004,0
4,0,3,0,1,1,0,5,10,1,303,1667,207,352,0,0,0,12513,2002,0


In [20]:
df_train_smote_y.shape

(6562438, 19)

In [ ]:
df_train, y_train = df_train_smote, y_train_smote

#### a quick reminder:
df_test_zindi = the test set from the "test" folder (no targets there)

df_entire = the entire dataset from the "train" folder (from which we dropped cases before 2005)

df_train = train subset from df_entire

df_test = test subset from df_entire

df = temporary random subset of df_train (for experemintation purposes)

## Feature Selection

### features to use:

* client_catg
* region
* tarif_type

* counter_number 
* counter_statue
* counter_code
* reading_remarqu
* counter_coefficient

* consommation_level_1 ... consommation_level_4 (baseline model)
* new_index - ?
* months_number
* counter_type


### features to omit:

* disrict (this information is dependent on 'region' and is "stored" there already)
* client_id
* creation_date
* invoice_date (feature engineer month / year?)
* old_index (almost perfectly correlates with 'new_index')


### features to engineer:

* month
* year
* ratios

## Subsample the train set for experiments

In [ ]:
df, _, y, _ = train_test_split(df_train, y_train, train_size=.1, stratify=y_train)
df.shape

## Feature engineering
#### (average consumption by year and month)



In [ ]:
cols = ['consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'invoice_year', 'invoice_month']

df_lookup = df[cols].groupby(by=['invoice_year', 'invoice_month']).mean()

In [ ]:
df_lookup.head(15)

In [ ]:
def lookup(sr, **kwargs):
    # ! cannot handle non Series
    # TODO if (2004, 12) not found - return what? - overall average for that month?
    # try: error
    sr = df_lookup.loc[tuple(sr)]
    return sr


# if this is the train df then:
nx = list(zip(df['invoice_year'], df['invoice_month']))
df_avg_consumption = df_lookup.loc[nx].reset_index(drop=True)

# the index of df_avg_consumption must be the same as df's
df_avg_consumption.index = df.index



""" commented this out because takes a long time - but will need this in the prediction/setting
# if df_test (or new data) then use the lookup function 
df_avg_consumption = df[['invoice_year', 'invoice_month']].apply(lookup, axis=1)
"""

df_avg_consumption.head(4)


In [ ]:
df_avg_consumption.shape

In [ ]:
cols = ['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4']

df_consumption_ratio = df[cols] / df_avg_consumption
df_consumption_ratio.columns = [f"consumption_ratio_{i}" for i in range(1,5)]
df_consumption_ratio.fillna(value=0.0, inplace=True)
df_consumption_ratio.head(5)

In [ ]:
# concatinate

df = pd.concat([df, df_consumption_ratio], axis=1)
df.head(5)

In [ ]:
# Explore visually

nx = np.random.permutation(df.index)[:10000]

col = 'consumption_ratio_1'

xx = df.loc[nx, col].values
xx = np.log(xx)

yy = y[nx].values

plt.plot(xx, yy, '.');


In [ ]:
cols = ['consumption_ratio_1',
       'consumption_ratio_2', 'consumption_ratio_3', 'consumption_ratio_4']


nx = np.random.permutation(df.index)[:10000]

yy = y[nx]


# trying different variants:   MIN is promising
temp = df.loc[nx, cols].min(axis=1)

xx = temp.values
xx = np.log(xx)



plt.plot(xx, yy, '.');

### Add that min engineered fetaure

In [ ]:
cols = ['consumption_ratio_1', 'consumption_ratio_2', 'consumption_ratio_3', 'consumption_ratio_4']
df['min'] = df[cols].max(axis=1)
df['max'] = df[cols].max(axis=1)
df['mu'] = df[cols].mean(axis=1)
df['std'] = df[cols].std(axis=1)



In [ ]:
cols = ['consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4']

df['min2'] = df[cols].max(axis=1)
df['max2'] = df[cols].max(axis=1)
df['mu2'] = df[cols].mean(axis=1)
df['std2'] = df[cols].std(axis=1)


df['ratio'] = (df['counter_number'] / df['new_index']).fillna(0).replace((-np.inf, np.inf), 0)
df['interaction'] = np.log( (df['counter_number'] * df['new_index']).fillna(0).replace((-np.inf, np.inf), 0)).fillna(0).replace((-np.inf, np.inf), 0)


In [ ]:
df.info()

In [ ]:
# function printing model report
def report(model, X,y):
    CV=3
    ytrue = y
    ypred = cross_val_predict(model, X,y, cv=CV)
    print(classification_report(ytrue, ypred))
    print("---"*20)
    
    # f1
    #f1 = cross_val_score(model, X,y, scoring='f1', cv=CV)
    #print("f1:", f1.round(3), f1.mean().round(3))

    # f2
    scorer = make_scorer(fbeta_score, beta=2)
    #f2 = cross_val_score(model, X,y, scoring=scorer, cv=CV)
    #print("f2:", f2.round(3), f2.mean().round(3))

    # AUC
    ppred = cross_val_predict(model, X,y, cv=CV, method='predict_proba')[:,-1]
    auc = roc_auc_score(ytrue, ppred)
    print("AUC =", auc)
    

## Trying Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

md = DecisionTreeClassifier()
md.fit(df,y)

sr_imp = pd.Series(md.feature_importances_, index=df.columns).sort_values(ascending=False)
sr_imp

In [ ]:
report(md, df, y)

In [ ]:
### SUBSET TREE
cols = sr_imp[:15].index
df_tr = df[cols]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

md = DecisionTreeClassifier()
md.fit(df_tr,y)

sr_imp = pd.Series(md.feature_importances_, index=df_tr.columns).sort_values(ascending=False)
sr_imp

In [ ]:
report(md, df_tr, y)

## Trying Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
md = RandomForestClassifier(n_estimators=20)
md.fit(df, y)

sr_imp = pd.Series(md.feature_importances_, index=df.columns).sort_values(ascending=False)
sr_imp

In [ ]:
report(md, df, y)

### Logistic Regression

In [ ]:
df.columns

In [ ]:
cols = ['consumption_ratio_1',
       'consumption_ratio_2', 'consumption_ratio_3', 'consumption_ratio_4',
       'max', 'mu', 'std', 'max2', 'mu1', 'std2', 'ratio', 'interaction']


cols = ['client_catg', 'region', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'new_index',
       'months_number', 'counter_type', 'year_created', 'invoice_year',
       'invoice_month', 'invoice_weekday']


df_lr = df#[cols]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

md = LogisticRegression(max_iter=300, penalty='l1', solver='liblinear')
sc = StandardScaler()
pl = make_pipeline(sc, md)
pl.fit(df_lr, y)



In [ ]:
sr_imp = pd.Series(pl.steps[1][1].coef_.ravel(), index=df_lr.columns).sort_values(key=abs, ascending=False)
sr_imp

In [ ]:
report(pl, df_lr, y)